In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('fivethirtyeight')
np.random.seed(113)

In [2]:
# loading csv file
df = pd.read_csv('../datasets/breast-cancer-wisconsin.data', names=['sample', 'thickness', 'size', 'shape', 'adhesion', 'epithelial', 'nuclei', 'chromatin', 'nucleoli', 'mitoses', 'status'], na_values='?')

print(df.isnull().sum())
df.head(10)

sample         0
thickness      0
size           0
shape          0
adhesion       0
epithelial     0
nuclei        16
chromatin      0
nucleoli       0
mitoses        0
status         0
dtype: int64


,sample,thickness,size,shape,adhesion,epithelial,nuclei,chromatin,nucleoli,mitoses,status
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2
5,1017122,8,10,10,8,7,10.0,9,7,1,4
6,1018099,1,1,1,1,2,10.0,3,1,1,2
7,1018561,2,1,2,1,2,1.0,3,1,1,2
8,1033078,2,1,1,1,2,1.0,1,1,5,2
9,1033078,4,2,1,1,2,1.0,2,1,1,2


In [3]:
y = df['status']
X = df.drop(columns=['status', 'sample'])
# 'status' goes for labels and number of 'sample' isn't important

print(X.dtypes)
X.head(10)

thickness       int64
size            int64
shape           int64
adhesion        int64
epithelial      int64
nuclei        float64
chromatin       int64
nucleoli        int64
mitoses         int64
dtype: object


,thickness,size,shape,adhesion,epithelial,nuclei,chromatin,nucleoli,mitoses
0,5,1,1,1,2,1.0,3,1,1
1,5,4,4,5,7,10.0,3,2,1
2,3,1,1,1,2,2.0,3,1,1
3,6,8,8,1,3,4.0,3,7,1
4,4,1,1,3,2,1.0,3,1,1
5,8,10,10,8,7,10.0,9,7,1
6,1,1,1,1,2,10.0,3,1,1
7,2,1,2,1,2,1.0,3,1,1
8,2,1,1,1,2,1.0,1,1,5
9,4,2,1,1,2,1.0,2,1,1


In [4]:
# cleaning
X.fillna(X.mean(), inplace=True)
# changing NaN values for mean values

print(X.isna().sum())

# making correction of types
X['nuclei'] = X['nuclei'].astype('int64')
print(X.dtypes)
X.head(10)

thickness     0
size          0
shape         0
adhesion      0
epithelial    0
nuclei        0
chromatin     0
nucleoli      0
mitoses       0
dtype: int64
thickness     int64
size          int64
shape         int64
adhesion      int64
epithelial    int64
nuclei        int64
chromatin     int64
nucleoli      int64
mitoses       int64
dtype: object


,thickness,size,shape,adhesion,epithelial,nuclei,chromatin,nucleoli,mitoses
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1
5,8,10,10,8,7,10,9,7,1
6,1,1,1,1,2,10,3,1,1
7,2,1,2,1,2,1,3,1,1
8,2,1,1,1,2,1,1,1,5
9,4,2,1,1,2,1,2,1,1


In [5]:
class CustomPerceptron(object):
     
    def __init__(self, n_iterations=100, random_state=1, learning_rate=0.01):
        self.n_iterations = n_iterations
        self.random_state = random_state
        self.learning_rate = learning_rate
 
    '''
    Stochastic Gradient Descent
     
    1. Weights are updated based on each training examples.
    2. Learning of weights can continue for multiple iterations
    3. Learning rate needs to be defined
    '''
    def fit(self, X, y):
        rgen = np.random.RandomState(self.random_state)
        self.coef_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []
        for _ in range(self.n_iterations):
            errors = 0
            for xi, expected_value in zip(X, y):
                predicted_value = self.predict(xi)
                self.coef_[1:] = self.coef_[1:] + self.learning_rate * (expected_value - predicted_value) * xi
                self.coef_[0] = self.coef_[0] + self.learning_rate * (expected_value - predicted_value) * 1
                update = self.learning_rate * (expected_value - predicted_value)
                errors += int(update != 0.0)
            self.errors_.append(errors)
    '''
    Net Input is sum of weighted input signals
    '''
    def net_input(self, X):
            weighted_sum = np.dot(X, self.coef_[1:]) + self.coef_[0]
            return weighted_sum
     
    '''
    Activation function is fed the net input and the unit step function
    is executed to determine the output.
    '''
    def activation_function(self, X):
            weighted_sum = self.net_input(X)
            return np.where(weighted_sum >= 0.0, 1, 0)
     
    '''
    Prediction is made on the basis of output of activation function
    '''
    def predict(self, X):
        return self.activation_function(X)
     
    '''
    Model score is calculated based on comparison of
    expected value and predicted value
    '''
    def score(self, X, y):
        misclassified_data_count = 0
        for xi, target in zip(X, y):
            output = self.predict(xi)
            if(target != output):
                misclassified_data_count += 1
        total_data_count = len(X)
        self.score_ = (total_data_count - misclassified_data_count)/total_data_count
        return self.score_

In [6]:
# declaration of perceptron class
class Perceptron:
    
    def __init__(self, n_dim):
        self.weights = 2 * np.random.random((n_dim, 1)) - 1
        self.n_dim   = n_dim
        
    def ReLu(self, x):
        return (1 / (1 + np.exp(-x)))[0]
    
    def ReLu_derivative(self, x):
        return (x * (1 - x))[0]
    
    def perceptronComputing(self, inputs):
        inputs  = inputs.astype(float)
        outputs = self.ReLu(np.dot(inputs, self.weights))
        
        return outputs
    
    def train(self, inputs, outputs, n_iter=1000):
        
        for i in range(n_iter):
            results = self.perceptronComputing(inputs)
            error   = outputs - results
            adjustments  = np.dot(inputs.T, error * self.ReLu_derivative(results))
            #print(self.weights, adjustments.T) #(self.weights.T + 
            self.weights += adjustments.reshape(self.n_dim,1)

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [8]:
n_dim = 3

# preprocessing
preprocesser = StandardScaler()
X = preprocesser.fit_transform(X)

# decomposition
#pca = PCA(n_components=n_dim)
#pca.fit(X)
#X   = pca.transform(X)

In [11]:
perceptron = Perceptron(X.shape[1])
print("Perceptron weights before training:\n", perceptron.weights)

y[y==2] = 0; y[y==4] = 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

perceptron.train(X_train, y_train, 1000)

Perceptron weights before training:
 [[ 0.11757155]
 [ 0.45834633]
 [-0.57658094]
 [-0.71982568]
 [ 0.38460923]
 [-0.36142816]
 [-0.73486186]
 [-0.43537103]
 [ 0.07990536]]


/var/folders/f9/1hf2kl857w37mlf3nbw3_jn40000gp/T/ipykernel_86950/543531545.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y==2] = 0; y[y==4] = 1


In [12]:
print("After training:\n", perceptron.weights)

After training:
 [[39.33693792]
 [44.15122655]
 [43.12700387]
 [36.80284047]
 [35.90779837]
 [44.22254093]
 [39.61299918]
 [37.65688766]
 [22.95648392]]


In [13]:
perceptron.perceptronComputing(X_test)

array([1.5513732e-98])